In [112]:
# Importing the necessary libraries
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

In [177]:
root_url = 'https://www.immoweb.be/en'

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36",
}

link = 'classified/flat-studio/for-sale/uccle/1180/20316229'


In [178]:
# Get the website's content and making soup

req = requests.get(f'{root_url}/{link}', headers=headers)
print(req.status_code)

content = req.content
soup = BeautifulSoup(content, 'html')


200


In [ ]:
# # Get overview details
# det_box = soup.find_all('section', attrs={'class': 'classified__section'})
# 
# overview = [tag.text for tag in det_box]
# 
# overview = [re.sub(r'[\s]', ' ', elem) for elem in overview]
# 
# overview

In [179]:
# City / Locality 

link = 'https://www.immoweb.be/en/classified/flat-studio/for-sale/uccle/1180/20316229'
locality = link.split('/')[-3]
locality

'uccle'

In [180]:
# Property Type
p_type = link.split('/')[-5]

p_type

'flat-studio'

In [13]:
################# To be done

# Subtype
print(p_type)
sub_type= ''

if 'apt' in p_type:
    sub_type = 'NA'   # or "NA" ?
elif "House" in p_type:
    p_type = 'House'
    sub_type = p_type   # or "NA" ?
elif 'Chalet' in p_type:
    p_type = 'House'
    sub_type = 'Chalet'
elif 'Bungalow' in p_type:
    p_type = 'House'
    sub_type = 'Bungalow'


print('Type:' + p_type)
print('Sub type: ' + sub_type)

apartment-block
Type:apartment-block
Sub type: 


In [181]:
# Price

price = soup.find('span', attrs={'class': 'sr-only'})
price = re.sub(r'[\D]', "", price.text)
price

'239000'

In [182]:
###################
# Sale Type TO BE DONE

sale_type = soup.find('span', attrs={'class': 'flag-list__text'})

sale_type



In [219]:
# Get table with all other property details
table = soup.find_all('table', class_='classified-table')

In [257]:
# Create list of raw data from the table
labels = [str(th.string) for th in soup.find_all('th', class_='classified-table__header')]
values = [str(td.string) for td in soup.find_all('td', class_='classified-table__data')]

# removing extra spaces
clean_labels = [label.strip() for label in labels]
clean_values = [value.strip() for value in values]

# Convert to dict:
details_dict = {label: value for label, value in zip(clean_labels, clean_values)}

details_dict


{'Available date': 'December 31 2024 - 12:00 AM',
 'Construction year': '1950',
 'Floor': '3',
 'Number of floors': '3',
 'Building condition': 'Good',
 'Number of frontages': '2',
 'Asbestos certificate is available': 'Not specified',
 'Living area': 'None',
 'Kitchen type': 'Installed',
 'Bathrooms': '1',
 'Shower rooms': '1',
 'Toilets': '1',
 'Furnished': 'No',
 'Primary energy consumption': 'None',
 'Energy class': 'E',
 'Reference number of the EPC report': 'Not specified',
 'CO₂ emission': '52 kg CO₂/m²',
 'Yearly theoretical total energy consumption': 'Not specified',
 'Conformity certification for fuel tanks': 'Not specified',
 'Heating type': 'Gas',
 'Double glazing': 'Yes',
 'Flood zone type': 'Non flood zone',
 'Price': 'None',
 'Tenement building': 'No',
 'Address': 'None',
 'Website': 'None',
 'External reference': '6328394'}

In [ ]:
# Searching for all details needed in details_dict and
# Assign "NA" to 

In [19]:
house = {'Locality' : '',
         'Type' : '',   # (House/apartment)
         'Subtype' : '', # (Bungalow, Chalet, Mansion, ...)
         'Price'  : '',
         'sale_type' : '',
         'bedrooms' : '',
         'living_area' : '',
         'equip_kit' : '',   # Yes, no
         'furnished' : '' , # Yes, no
         'open_fire' : '', # Yes, no
         'terrace' : '', # Yes, no  # If yes, area: 
         'garden' : '', # Yes, no  # If yes, area: 
         'land_surface' : '',
         # Surface area of the plot of land (?)
         'facades' : '',
         'swim_pool' : '',  # Yes, no
         'state' : ''     #  (New, to be renovated, ...)
          
                  }

In [115]:
def get_details(link) -> dict:
    """
    Takes a link for a listing and returns its details.

    PARAMS:
    link - str: the url of the property listing to scrap

    RETURNS:
    house - dict: label-value pairs of the relevant property details

    """

    # Headers for our req so it does not get blocked
    headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36",
        }

    # Making the request and printing status code
    req = requests.get(link, headers=headers)

    if req.status_code != 200:
        print("Unable to process")
    else:

        # Get content and make soup
        content = req.content
        soup = BeautifulSoup(content, features='lxml')

        # Iniciate an empty dict
        house = {}

        # Fetch the html tags that will be used to extract the info
        price = soup.find('span', attrs={'class': 'sr-only'}).text
        #table = soup.find_all('table', class_='classified-table')

        # Convert table to lists
        labels = [th for th in soup.find_all('th', class_='classified-table__header')]
        values = [td for td in soup.find_all('td', class_='classified-table__data')]

        # Convert lists to df
        # df = pd.DataFrame({
        # 'Headers' : labels,
        # 'Values' : values
        #     })

        # Populate house dict with the info
        house['id'] = link.split('/')[-1]
        house['locality'] = link.split('/')[-3]
        house['p_type'] = link.split('/')[-5]
        house['price'] = re.sub(r'[\D]', "", price)

        return house


In [29]:
links = ['https://www.immoweb.be/en/classified/apartment/for-sale/antwerpen/2100/20315980', 'https://www.immoweb.be/en/classified/house/for-sale/jette/1090/20231308', 'https://www.immoweb.be/en/classified/country-cottage/for-sale/vaux-sur-sure/6640/20057933']

In [116]:
def make_list(file):
    """ 
    Gets the details of each property and creates a dictionary for it.
    
    PARAMS:
    links_list -list: of the links to extract the info from

    RETURN
    details -list: of dictionaries
    """

    links = []
    links_file = open(file, 'r')
    for link in links_file.readlines():
        links.append(link)

    details = [get_details(link) for link in links]

    df = pd.DataFrame(details)
    df.to_csv('Test.csv', index=False)

    return details

In [50]:
def convert(details):
    """
    Converts the list of dictionaries to
    a dataframe and saves it as csv file.

    PARAMS:
    details -list: of dictionaries

    RETURN:
    df: dataFrame saved in as csv file
    """

    df = pd.DataFrame(details)
    df.to_csv('Test.csv', index=False)
    return df

In [131]:
make_list('immo_links.txt')

[{'id': '20311767\n',
  'locality': 'seraing',
  'p_type': 'mixed-use-building',
  'price': '169900'},
 {'id': '20307293\n',
  'locality': 'nieuwerkerken',
  'p_type': 'house',
  'price': '299000'},
 {'id': '20311659\n',
  'locality': 'ninove',
  'p_type': 'house',
  'price': '315000'},
 {'id': '20311730\n',
  'locality': 'houthalen-helchteren',
  'p_type': 'house',
  'price': '299500'},
 {'id': '20311729\n',
  'locality': 'gent',
  'p_type': 'house',
  'price': '595000'},
 {'id': '20311728\n',
  'locality': 'blankenberge',
  'p_type': 'manor-house',
  'price': '610000'},
 {'id': '20311727\n',
  'locality': 'blankenberge',
  'p_type': 'duplex',
  'price': '275000'},
 {'id': '20311726\n',
  'locality': 'merchtem',
  'p_type': 'house',
  'price': '429000'},
 {'id': '20311725\n',
  'locality': 'hoboken%20%28antwerpen%29',
  'p_type': 'apartment',
  'price': '219000'},
 {'id': '20311723\n',
  'locality': 'braine-l%27alleud',
  'p_type': 'house',
  'price': '250000'},
 {'id': '20311722\n',
